In [2]:
import pandas as pd
import mlflow
from sklearn.datasets import load_iris

In [3]:
#load dataset
iris_data=load_iris()
print(iris_data.keys())
df_iris = pd.DataFrame(iris_data['data'],columns=iris_data['feature_names'])
df_iris.head()


dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
df_iris['target'] = iris_data['target'] # added new column to the dataframe
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
X=df_iris.iloc[:,0:4]
Y=df_iris.iloc[:,4:]
print('shape of x ',X.shape)
print('shape of y ',Y.shape)

shape of x  (150, 4)
shape of y  (150, 1)


In [6]:
#split the data into training and testing 
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest =train_test_split(X,Y,test_size=0.30,random_state=1)
print('xtrain shape',xtrain.shape)
print('xtest shape',xtest.shape)
print('ytrain shape',ytrain.shape)
print('ytest shape',ytest.shape)


xtrain shape (105, 4)
xtest shape (45, 4)
ytrain shape (105, 1)
ytest shape (45, 1)


In [7]:
from sklearn.linear_model import LogisticRegression
model_param = {
    "penalty":"l2",
    "solver":"saga",
    "random_state":11
    
}
model =LogisticRegression(**model_param)
model


LogisticRegression(random_state=11, solver='saga')

In [8]:
import numpy as np
print(type(np.array(ytrain['target']).ravel()))
ytrain1=np.array(ytrain['target']).ravel()
model.fit(xtrain,ytrain1)



<class 'numpy.ndarray'>


e:\mlflowdemo\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(random_state=11, solver='saga')

In [9]:
from sklearn.metrics import accuracy_score
ypred = model.predict(xtest)
ypred

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 2,
       1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2,
       1])

In [10]:
accuracyscore = accuracy_score(ytest,ypred)
print(accuracyscore)

0.9777777777777777


In [11]:
# mlflow tracking 
uri ="http://127.0.0.1:5000"
mlflow.set_tracking_uri(uri)
experimentName ='iris_logistic_exp'
mlflow.set_experiment(experiment_name=experimentName)

<Experiment: artifact_location='mlflow-artifacts:/547324179472204710', creation_time=1745687851113, experiment_id='547324179472204710', last_update_time=1745687851113, lifecycle_stage='active', name='iris_logistic_exp', tags={}>

In [17]:
#start mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
with mlflow.start_run():
    mlflow.log_params(model_param)
    mlflow.log_metric("accuracyscore",accuracyscore)
    #tag is use for user to check what they did
    mlflow.set_tag("Experiment Info","Logistic model for iris dataset 1")
    #infer model signature 
    sign = infer_signature(xtrain,model.predict(xtrain))
    mlflow.log_param("model_name",model)
    modelinfo=  mlflow.sklearn.log_model(model,"iris_datamodel",input_example=xtrain,signature=sign,
                             registered_model_name="tracking_modeel")
    
   



Registered model 'tracking_modeel' already exists. Creating a new version of this model...
2025/04/27 23:13:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_modeel, version 3


🏃 View run adaptable-zebra-587 at: http://127.0.0.1:5000/#/experiments/547324179472204710/runs/d343466e1cf54961a5243aa86822766c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/547324179472204710


Created version '3' of model 'tracking_modeel'.


In [19]:
from mlflow.models import validate_serving_input
modelinfo.model_uri

'runs:/d343466e1cf54961a5243aa86822766c/iris_datamodel'

In [ ]:
#validating model
model_uri = modelinfo.model_uri
# the model is logged with input example
# mlflow converts it in to  payload format for deployed endpoint 
# saves it to payload.json
# import json 
#json.loads() ,load()
payload="""{
  "dataframe_split": {
    "columns": [
      "sepal length (cm)",
      "sepal width (cm)",
      "petal length (cm)",
      "petal width (cm)"
    ],
    "data": [
      [
        7.7,
        2.6,
        6.9,
        2.3
      ],
      [
        5.7,
        3.8,
        1.7,
        0.3
      ],
      [
        5,
        3.6,
        1.4,
        0.2
      ],
      [
        4.8,
        3,
        1.4,
        0.3
      ],
      [
        5.2,
        2.7,
        3.9,
        1.4
      ],
      [
        5.1,
        3.4,
        1.5,
        0.2
      ],
      [
        5.5,
        3.5,
        1.3,
        0.2
      ],
      [
        7.7,
        3.8,
        6.7,
        2.2
      ],
      [
        6.9,
        3.1,
        5.4,
        2.1
      ],
      [
        7.3,
        2.9,
        6.3,
        1.8
      ],
      [
        6.4,
        2.8,
        5.6,
        2.2
      ],
      [
        6.2,
        2.8,
        4.8,
        1.8
      ],
      [
        6,
        3.4,
        4.5,
        1.6
      ],
      [
        7.7,
        2.8,
        6.7,
        2
      ],
      [
        5.7,
        3,
        4.2,
        1.2
      ],
      [
        4.8,
        3.4,
        1.6,
        0.2
      ],
      [
        5.7,
        2.5,
        5,
        2
      ],
      [
        6.3,
        2.7,
        4.9,
        1.8
      ],
      [
        4.8,
        3,
        1.4,
        0.1
      ],
      [
        4.7,
        3.2,
        1.3,
        0.2
      ],
      [
        6.5,
        3,
        5.8,
        2.2
      ],
      [
        4.6,
        3.4,
        1.4,
        0.3
      ],
      [
        6.1,
        3,
        4.9,
        1.8
      ],
      [
        6.5,
        3.2,
        5.1,
        2
      ],
      [
        6.7,
        3.1,
        4.4,
        1.4
      ],
      [
        5.7,
        2.8,
        4.5,
        1.3
      ],
      [
        6.7,
        3.3,
        5.7,
        2.5
      ],
      [
        6,
        3,
        4.8,
        1.8
      ],
      [
        5.1,
        3.8,
        1.6,
        0.2
      ],
      [
        6,
        2.2,
        4,
        1
      ],
      [
        6.4,
        2.9,
        4.3,
        1.3
      ],
      [
        6.5,
        3,
        5.5,
        1.8
      ],
      [
        5,
        2.3,
        3.3,
        1
      ],
      [
        6.3,
        3.3,
        6,
        2.5
      ],
      [
        5.5,
        2.5,
        4,
        1.3
      ],
      [
        5.4,
        3.7,
        1.5,
        0.2
      ],
      [
        4.9,
        3.1,
        1.5,
        0.2
      ],
      [
        5.2,
        4.1,
        1.5,
        0.1
      ],
      [
        6.7,
        3.3,
        5.7,
        2.1
      ],
      [
        4.4,
        3,
        1.3,
        0.2
      ],
      [
        6,
        2.7,
        5.1,
        1.6
      ],
      [
        6.4,
        2.7,
        5.3,
        1.9
      ],
      [
        5.9,
        3,
        5.1,
        1.8
      ],
      [
        5.2,
        3.5,
        1.5,
        0.2
      ],
      [
        5.1,
        3.3,
        1.7,
        0.5
      ],
      [
        5.8,
        2.7,
        4.1,
        1
      ],
      [
        4.9,
        3.1,
        1.5,
        0.1
      ],
      [
        7.4,
        2.8,
        6.1,
        1.9
      ],
      [
        6.2,
        2.9,
        4.3,
        1.3
      ],
      [
        7.6,
        3,
        6.6,
        2.1
      ],
      [
        6.7,
        3,
        5.2,
        2.3
      ],
      [
        6.3,
        2.3,
        4.4,
        1.3
      ],
      [
        6.2,
        3.4,
        5.4,
        2.3
      ],
      [
        7.2,
        3.6,
        6.1,
        2.5
      ],
      [
        5.6,
        2.9,
        3.6,
        1.3
      ],
      [
        5.7,
        4.4,
        1.5,
        0.4
      ],
      [
        5.8,
        2.7,
        3.9,
        1.2
      ],
      [
        4.5,
        2.3,
        1.3,
        0.3
      ],
      [
        5.5,
        2.4,
        3.8,
        1.1
      ],
      [
        6.9,
        3.1,
        4.9,
        1.5
      ],
      [
        5,
        3.4,
        1.6,
        0.4
      ],
      [
        6.8,
        2.8,
        4.8,
        1.4
      ],
      [
        5,
        3.5,
        1.6,
        0.6
      ],
      [
        4.8,
        3.4,
        1.9,
        0.2
      ],
      [
        6.3,
        3.4,
        5.6,
        2.4
      ],
      [
        5.6,
        2.8,
        4.9,
        2
      ],
      [
        6.8,
        3.2,
        5.9,
        2.3
      ],
      [
        5,
        3.3,
        1.4,
        0.2
      ],
      [
        5.1,
        3.7,
        1.5,
        0.4
      ],
      [
        5.9,
        3.2,
        4.8,
        1.8
      ],
      [
        4.6,
        3.1,
        1.5,
        0.2
      ],
      [
        5.8,
        2.7,
        5.1,
        1.9
      ],
      [
        4.8,
        3.1,
        1.6,
        0.2
      ],
      [
        6.5,
        3,
        5.2,
        2
      ],
      [
        4.9,
        2.5,
        4.5,
        1.7
      ],
      [
        4.6,
        3.2,
        1.4,
        0.2
      ],
      [
        6.4,
        3.2,
        5.3,
        2.3
      ],
      [
        4.3,
        3,
        1.1,
        0.1
      ],
      [
        5.6,
        3,
        4.1,
        1.3
      ],
      [
        4.4,
        2.9,
        1.4,
        0.2
      ],
      [
        5.5,
        2.4,
        3.7,
        1
      ],
      [
        5,
        2,
        3.5,
        1
      ],
      [
        5.1,
        3.5,
        1.4,
        0.2
      ],
      [
        4.9,
        3,
        1.4,
        0.2
      ],
      [
        4.9,
        2.4,
        3.3,
        1
      ],
      [
        4.6,
        3.6,
        1,
        0.2
      ],
      [
        5.9,
        3,
        4.2,
        1.5
      ],
      [
        6.1,
        2.9,
        4.7,
        1.4
      ],
      [
        5,
        3.4,
        1.5,
        0.2
      ],
      [
        6.7,
        3.1,
        4.7,
        1.5
      ],
      [
        5.7,
        2.9,
        4.2,
        1.3
      ],
      [
        6.2,
        2.2,
        4.5,
        1.5
      ],
      [
        7,
        3.2,
        4.7,
        1.4
      ],
      [
        5.8,
        2.7,
        5.1,
        1.9
      ],
      [
        5.4,
        3.4,
        1.7,
        0.2
      ],
      [
        5,
        3,
        1.6,
        0.2
      ],
      [
        6.1,
        2.6,
        5.6,
        1.4
      ],
      [
        6.1,
        2.8,
        4,
        1.3
      ],
      [
        7.2,
        3,
        5.8,
        1.6
      ],
      [
        5.7,
        2.6,
        3.5,
        1
      ],
      [
        6.3,
        2.8,
        5.1,
        1.5
      ],
      [
        6.4,
        3.1,
        5.5,
        1.8
      ],
      [
        6.3,
        2.5,
        4.9,
        1.5
      ],
      [
        6.7,
        3.1,
        5.6,
        2.4
      ],
      [
        4.9,
        3.6,
        1.4,
        0.1
      ]
    ]
  }
}"""
validate_serving_input(model_uri,payload)

array([2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0,
       2, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 0, 2, 0, 2, 2, 2, 0,
       0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 2,
       2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 0])

In [ ]:
#load the model back for prediction as generic python function model
loggedmodel = mlflow.pyfunc.load_model(modelinfo.model_uri)
print(loggedmodel)
predictions =loggedmodel.predict(xtest)
iris_test_result=pd.DataFrame(xtest,columns=iris_data['feature_names'])
iris_test_result['actualdata']=ytest
iris_test_result['predictedvalues']=predictions
iris_test_result.head()


mlflow.pyfunc.loaded_model:
  artifact_path: iris_datamodel
  flavor: mlflow.sklearn
  run_id: d343466e1cf54961a5243aa86822766c



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actualdata,predictedvalues
14,5.8,4.0,1.2,0.2,0,0
98,5.1,2.5,3.0,1.1,1,1
75,6.6,3.0,4.4,1.4,1,1
16,5.4,3.9,1.3,0.4,0,0
131,7.9,3.8,6.4,2.0,2,2
